**Uncapacited Lot-Sizing**

*Mixed Integer Programming Formulations*

\\section*{Uncapacitated Lot-Sizing (ULS)}

\textbf{Variáveis de decisão:}
\begin{itemize}
    \item y_t: quantidade produzida no período t
    \item s_t: estoque no final do período t
    \item x_t: variável binária que indica se há produção no período t (x_t = 1 se y_t > 0, caso contrário x_t = 0)
\end{itemize}

\textbf{Parâmetros:}
\begin{itemize}
    \item d_t: demanda no período t
    \item p_t: custo de produção por unidade no período t
    \item h_t: custo de armazenagem por unidade no período t
    \item f_t: custo fixo de produção no período t
    \item M: quantidade máxima que pode ser produzida no período (grande o suficiente)
\end{itemize}

\textbf{Formulação:}

\begin{align}
\min \quad & \sum_{t=1}^{n} p_t y_t + \sum_{t=1}^{n} h_t s_t + \sum_{t=1}^{n} f_t x_t \tag{1} \\
\text{sujeito a:} \quad & s_{t-1} + y_t = d_t + s_t \quad && \text{para } t = 1, \ldots, n \tag{2} \\
& y_t \leq M x_t \quad && \text{para } t = 1, \ldots, n \tag{3} \\
& s_0 = 0, \quad s_t \geq 0, \quad y_t \geq 0, \quad x_t \in \{0,1\} \quad && \text{para } t = 1, \ldots, n \tag{4}
\end{align}



In [ ]:
#Modelo de Programação Dinâmica
#Instâncias utilizadas do Livro do Wolsey-Segunda Edição (Página 82,Exemplo 5.1)

!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB

# Dados
n = 4
d = [2, 4, 5, 1]
p = [3, 3, 3, 3]
h = [1, 2, 1, 1]
f = [12, 20, 16, 8]
M = sum(d)  # Limite superior para produção

# Modelo
model = gp.Model("ULS")

# Variáveis
x = model.addVars(n, vtype=GRB.INTEGER, name="x")   # produção
I = model.addVars(n, vtype=GRB.INTEGER, name="I")   # inventário
y = model.addVars(n, vtype=GRB.BINARY,  name="y")   # setup

# Função objetivo
model.setObjective(
    gp.quicksum(f[t] * y[t] + p[t] * x[t] + h[t] * I[t] for t in range(n)),
    GRB.MINIMIZE
)

# Restrições de balanço de estoque
for t in range(n):
    if t == 0:
        model.addConstr(x[t] == d[t] + I[t])
    else:
        model.addConstr(I[t-1] + x[t] == d[t] + I[t])

# Restrições de ativação
for t in range(n):
    model.addConstr(x[t] <= M * y[t])

# Otimização
model.optimize()

# Resultados
if model.status == GRB.OPTIMAL:
    print(f"\nCusto ótimo: {model.ObjVal}")
    for t in range(n):
        print(f"Período {t+1}: Produção = {x[t].X}, Estoque = {I[t].X}, Setup = {int(y[t].X)}")
else:
    print("Nenhuma solução ótima encontrada.")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 12 columns and 19 nonzeros
Model fingerprint: 0x05a03e78
Variable types: 0 continuous, 12 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 92.0000000
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 6 rows, 10 columns, 15 nonzeros
Variable types: 0 continuous, 10 integer (3 binary)

Root relaxation: objective 5.933333e+01, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   59.33333   

**Relaxação Linear do Problema de Uncapacitated Lot-Sizing (ULS)**

\textbf{Parâmetros:}
\begin{itemize}
    \item n: número de períodos
    \item d_t: demanda no período t
    \item p_t: custo unitário de produção no período t
    \item h_t: custo unitário de armazenamento no período t
    \item f_t: custo fixo de produção no período t
    \item M: constante suficientemente grande
\end{itemize}

\textbf{Variáveis de decisão:}
\begin{itemize}
    \item x_t \: quantidade produzida no período t
    \item s_t \: estoque ao final do período t
    \item y_t \: variável de ativação contínua (relaxada)
\end{itemize}

\textbf{Função objetivo:}
\begin{equation}
\min \sum_{t=1}^{n} p_t x_t + h_t s_t + f_t y_t
\end{equation}

\textbf{Restrições:}
\begin{align}
s_{t-1} + x_t &= d_t + s_t, & \forall t = 1, \ldots, n \tag{Balanço de estoque} \\
x_t &\leq M y_t, & \forall t = 1, \ldots, n \tag{Relaxação da ativação} \\
s_0 &= 0 & \tag{Estoque inicial}
\end{align}

\textbf{Domínio das variáveis:}
\begin{align}
x_t &\geq 0, \quad s_t \geq 0, \quad 0 \leq y_t \leq 1, & \forall t = 1, \ldots, n
\end{align}
OBS: Setup fracionável, mas produção e estoque inteiros


In [ ]:
#Modelo de Programação Dinâmica(Utilizando Técnica de Relaxação Linear Inteira)
#Instâncias utilizadas do Livro do Wolsey-Segunda Edição(Página 82, Exemplo 5.1)
import gurobipy as gp
from gurobipy import GRB

# Dados
n = 4
d = [2, 4, 5, 1]
p = [3, 3, 3, 3]
h = [1, 2, 1, 1]
f = [12, 20, 16, 8]
M = sum(d)  # 12

# Criação do modelo
model = gp.Model("ULS_Relaxacao_Linear_Inteira")

# Variáveis
x = model.addVars(n, vtype=GRB.INTEGER, name="x")       # produção (inteira)
I = model.addVars(n, vtype=GRB.INTEGER, name="I")       # estoque (inteiro)
y = model.addVars(n, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="y")  # setup (contínua)

# Função objetivo
model.setObjective(
    gp.quicksum(f[t] * y[t] + p[t] * x[t] + h[t] * I[t] for t in range(n)),
    GRB.MINIMIZE
)

# Restrições de balanço de estoque
for t in range(n):
    if t == 0:
        model.addConstr(x[t] == d[t] + I[t], name=f"estoque_{t}")
    else:
        model.addConstr(I[t-1] + x[t] == d[t] + I[t], name=f"estoque_{t}")

# Restrições de ativação
for t in range(n):
    model.addConstr(x[t] <= M * y[t], name=f"setup_{t}")

# Otimização
model.optimize()

# Resultados
if model.status == GRB.OPTIMAL:
    print(f"\n Custo ótimo da relaxação linear inteira: {model.ObjVal:.2f}")
    for t in range(n):
        print(f"Período {t+1}: Produção = {x[t].X}, Estoque = {I[t].X}, Setup (y) = {y[t].X:.4f}")
else:
    print("Nenhuma solução ótima encontrada.")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 12 columns and 19 nonzeros
Model fingerprint: 0x2ed07745
Variable types: 4 continuous, 8 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolved: 3 rows, 7 columns, 9 nonzeros
Variable types: 0 continuous, 7 integer (0 binary)
Found heuristic solution: objective 53.3333333
Found heuristic solution: objective 53.0000000

Root relaxation: objective 5.200000e+01, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap